In [2]:
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import calendar
import undetected_chromedriver as uc
import locale

In [ ]:

json_pages_info = {"pages": []}

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

articles_count = 0

pages = 5

url = f"https://www.latercera.com/search/?q=Estallido%20Social%20Chile"

driver.get(url)

time.sleep(10)

for page in range(2, pages + 1):

	elements = driver.find_element(By.CLASS_NAME, "gsc-expansionArea")
	articles = elements.find_elements(By.CLASS_NAME, "gsc-webResult")
	print(f"{len(articles)} articles found")

	for article in articles:
		print("Analyzing article...")

		date = article.find_element(By.CLASS_NAME, "gsc-table-result").find_element(By.CLASS_NAME, "gsc-table-cell-snippet-close").find_element(By.CLASS_NAME, "gs-bidi-start-align")
		print(date.text)
		originalDate = date.text[:10] if date else "not found"
		splitDate = originalDate.split("-")

		if(len(splitDate) != 3 or date == None):
			print("Fecha no encontrada")
			continue
		
		timestamp = calendar.timegm(time.strptime(f'{splitDate[2]}-{splitDate[1]}-{splitDate[0]} 00:00:00', '%Y-%m-%d %H:%M:%S'))

		if (timestamp < 1573786800 or timestamp > 1702782000):
			print("La noticia no corresponde a la fecha solicitada")
			continue

		print(splitDate)

		try:

			link = article.find_element(By.CLASS_NAME, "gs-title").find_element(By.CLASS_NAME, "gs-title").get_attribute("href")
			title = article.find_element(By.CLASS_NAME, "gs-title").find_element(By.CLASS_NAME, "gs-title").text
			image_link = article.find_element(By.CLASS_NAME, "gs-image").find_element(By.CLASS_NAME, "gs-image").get_attribute("src")
			description = article.find_element(By.CLASS_NAME, "gsc-table-result").find_element(By.CLASS_NAME, "gsc-table-cell-snippet-close").find_element(By.CLASS_NAME, "gs-bidi-start-align")
			description = description.text[10:] if description else "not found"
		except Exception as e:
			print("Error al obtener los datos del artículo: ", e)
			continue

		link_info = {
			"newscast" : "La Tercera",
			"title": title,
			"description": description,
			"category": "The site does not provide a category",
			"date": originalDate,
			"image_link": image_link,
			"author": "not found initially",
			"link": link,
		}
		articles_count += 1
		print("Añadiendo articulo")
		json_pages_info["pages"].append(link_info)

	buttons = driver.find_element(By.CLASS_NAME, "gsc-cursor")
	buttons = buttons.find_elements(By.CLASS_NAME, "gsc-cursor-page")
	print(len(buttons))

	button_found = False

	for button in buttons:
		if(button.text == str(page)):
			print("Button found: " + button.text)
			button.click()
			button_found = True
			time.sleep(2)
			break

	if(not button_found):
			print("The button for page " + str(page) + " was not found.")
			break

driver.quit()

with open(f"../archive/temp/laTercera_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


In [15]:
import pyautogui
import random

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

# json_pages_info = {"pages": []}

pages = 32

data_extracted = True

re = False

for page in range(22, pages):

	first = True
	while (re or first):
		first = False
		
		try: 
			time.sleep(1)

			if (not re): 
				options = uc.ChromeOptions()
				options.add_argument("--no-sandbox")
				options.add_argument("--disable-blink-features=AutomationControlled")

				driver = uc.Chrome(options=options)

				url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Awww.latercera.com&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'
				driver.get(url)

			re = False
			time.sleep(random.uniform(6, 12))

			articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
			articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

			location = articles_section.location
			size = articles_section.size
			x = location['x'] + random.randint(5, size['width'] - 5)
			y = location['y'] + random.randint(5, size['height'] - 5)
			duration = random.uniform(0.8, 2.0)
			pyautogui.moveTo(x, y, duration=duration)

			for article in articles:
				print("revisando artículo...")
				try: 
					originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
					for es, en in months.items():
						if es in originalDate:
								originalDate = originalDate.replace(es, en)
								break
						
					date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

					if (date_epoch < 1573786800 or date_epoch > 1702782000):
						print("La noticia no corresponde a la fecha solicitada")
						continue

					try:
						title = article.find_element(By.TAG_NAME, "h3").text
						description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
						link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

						link_info = {
							"newscast" : "El Mostrador",
							"title": title,
							"description": description,
							"category": "The site does not provide a category",
							"date": originalDate,
							"image_link": "not found initially",
							"author": "not found initially",
							"link": link,
						}

						print("información incluida!")
						json_pages_info["pages"].append(link_info)

					except Exception as e:
						print(f"Error al extraer información del artículo: {e}")
						continue

				except Exception as e:
					print(f"Error al procesar el artículo: {e}")
					continue
		except Exception as e:
			print(f"Error al procesar el resultado de busqueda. Página: {page}")
			print(f"Sitio: {url}")
			print(e)
			response = input("Reintentar? y/n")
			if response.lower() == 'y':
				print("Reintentando.")
				re = True
			else: 
				print("Cancelando...")
				re = False
		
		if (not re):
			driver.quit()

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/la_tercera_080725.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


# with open(f"output.html", "w", encoding="utf-8") as f:
# 	f.write(driver.page_source)

revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
información incluida!
revisando artículo...
Error al procesar el artículo: Message: no such element: Unable to locate element: {"method":"css selector","selector":".YrbPuc"}
  (Session info: chrome=138.0.7204.97); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0xb744a3+62419]
	GetHandleVerifier [0x0xb744e4+62484]
	(No symbol) [0x0x9b2133]
	(No symbol) [0x0x9fa8fe]
	(No symbol) [0x0x9fac9b]
	(No symbol) [0x0x9f0131]
	(No symbol) [0x0xa1f4b4]
	(No 

In [ ]:
save_data = {}

# with open(f"../archive/temp/la_tercera_080725.json", 'w', encoding='utf-8') as file:
# 	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)

with open(f"../archive/temp/la_tercera_080725_01.json", 'r', encoding='utf-8') as file:
	save_data = json.load(file)

save_data["pages"] = save_data["pages"] + json_pages_info["pages"]


Almacenando información en el archivo JSON...


In [ ]:
import json

all_data = {"pages": []}
coincidence = 0
clean_data_links = []

clean_data = {
	"pages": []
}

with open("../archive/temp/la_tercera_080725_raw.json", 'r', encoding='utf-8') as file:
	all_data = json.load(file)

all_links = [page["link"] for page in all_data["pages"]]

coincidence_boolean = False

print(f"Total links in raw data: {len(all_data['pages'])}")

for page in all_data["pages"]:
	for link in clean_data_links:
		if page["link"] in link:
			print(f"Coincidencia encontrada: {page['link']} con link {link}")
			coincidence += 1
			coincidence_boolean = True
			break
	if not coincidence_boolean:
		clean_data["pages"].append(page)
		clean_data_links.append(page["link"])
	coincidence_boolean = False

print(f"Total coincidences: {coincidence}")
print(f"Total unique links: {len(clean_data['pages'])}")

with open(f"../archive/temp/la_tercera_080725_cleanData.json", 'w', encoding='utf-8') as file:
	json.dump(clean_data, file, ensure_ascii=False, indent=4)

Total links in raw data: 309
Coincidencia encontrada: https://www.latercera.com/culto/2020/07/27/mario-toral-el-estallido-social-nos-hizo-abrir-los-ojos/ con link https://www.latercera.com/culto/2020/07/27/mario-toral-el-estallido-social-nos-hizo-abrir-los-ojos/
Coincidencia encontrada: https://www.latercera.com/la-tercera-sabado/noticia/llagas-del-18-0-similitudes-y-contrastes-en-santiago-y-valparaiso/GR6EZ2VTV5C4TIHMHY4GQQ6KMU/ con link https://www.latercera.com/la-tercera-sabado/noticia/llagas-del-18-0-similitudes-y-contrastes-en-santiago-y-valparaiso/GR6EZ2VTV5C4TIHMHY4GQQ6KMU/
Coincidencia encontrada: https://www.latercera.com/nacional/noticia/efecto-pandemia-prensa-escrita-medios-online-y-la-tv-repuntan-en-confianza-y-redes-sociales-caen-fuertemente-en-su-evaluacion/XKWPNE4VQVBYTHHZDF5IDZCPKM/ con link https://www.latercera.com/nacional/noticia/efecto-pandemia-prensa-escrita-medios-online-y-la-tv-repuntan-en-confianza-y-redes-sociales-caen-fuertemente-en-su-evaluacion/XKWPNE4VQVB